#### Mixing in Columns

The _most basic way_ to __specify a column on a mixin__ is by _simple declaration_.

In [1]:
from sqlalchemy import Column, Integer, String, DateTime, ForeignKey, func
from sqlalchemy.orm import declared_attr, declarative_base, declarative_mixin

In [2]:
Base = declarative_base()

In [3]:
@declarative_mixin
class TimestampMixin:
    created_at = Column(DateTime, default=func.now())

In [4]:
class MyModel(TimestampMixin, Base):
    __tablename__ = "test"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(1000))

Where above, _all declarative classes_ that include `TimestampMixin` will also have a column `created_at` that __applies a timestamp to all row insertions__.

Those familiar with the `SQLAlchemy expression language` know that the __object identity of clause elements defines their role in a schema__. Two `Table` objects _a and b may both have a column_ called `id`, but the way these are __differentiated__ is that `a.c.id` and `b.c.id` are _two distinct Python objects_, __referencing__ their _parent tables a and b_ respectively.

In the case of the _mixin column_, it seems that _only one_ `Column` object is __explicitly created__, yet the _ultimate_ `created_at` column above __must exist__ as a _distinct Python object for each separate destination class_. To accomplish this, the _declarative extension_ __creates a copy of each `Column` object__ encountered on a class that is detected as a _mixin_.

This _copy mechanism_ is __limited to simple columns__ that have __no foreign keys__, as a `ForeignKey` itself _contains references to columns_ which __can't be properly recreated__ at this level. For _columns that have foreign keys_, as well as for the _variety of mapper-level constructs_ that __require destination-explicit context__, the `declared_attr` decorator is provided so that _patterns common to many classes_ __can be defined as `callables`__.

In [5]:
@declarative_mixin
class ReferenceAddressMixin:
    @declared_attr
    def address_id(cls):
        return Column(Integer, ForeignKey("address.id"))

In [6]:
class User(ReferenceAddressMixin, Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)

Where above, the `address_id` _class-level callable_ is __executed at the point__ at which the `User` class is __constructed__, and the _declarative extension_ __can use the resulting `Column` object__ as returned by the method __without the need to copy it__.

`Columns` _generated by_ `declared_attr` can also be _referenced by_ `__mapper_args__` to a __limited degree__, currently by `polymorphic_on` and `version_id_col`; the _declarative extension_ will __resolve__ them at _class construction time_.

In [7]:
@declarative_mixin
class MyMixin:
    @declared_attr
    def type_(cls):
        return Column(String(50))
    
    __mapper_args__ = {"polymorphic_on": type_}

In [8]:
class PolymorphicModel(MyMixin, Base):
    __tablename__ = "polymorphic_table"
    id = Column(Integer, primary_key=True)